## Tiny LMP - MNIST/FashionMNIST classification

In [1]:
import torch
import torchvision
from torchvision import transforms
import numpy as np
from PIL import Image
from tvm import relay
import onnx

# Tiny fully-connected network
class TinyMLP(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = torch.nn.Linear(28*28, 128)
        self.fc2 = torch.nn.Linear(128, 10)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

tiny_mlp = TinyMLP().eval()

# Example input
x_mlp = torch.randn(1, 1, 28, 28)

# Forward pass (optional check)
y_mlp = tiny_mlp(x_mlp)
print("Tiny MLP output shape:", y_mlp.shape)

# Trace and convert to TVM Relay
scripted_mlp = torch.jit.trace(tiny_mlp, x_mlp).eval()
mod_mlp, params_mlp = relay.frontend.from_pytorch(scripted_mlp, [("input0", x_mlp.shape)])
print("✅ Tiny MLP successfully converted to TVM Relay")


Tiny MLP output shape: torch.Size([1, 10])
✅ Tiny MLP successfully converted to TVM Relay


In [2]:
import tvm
from tvm.contrib import graph_executor

# Choose your target architecture
# Example: x86 CPU
target_str = "llvm -mtriple=x86_64-linux-gnu -mcpu=haswell"
target = tvm.target.Target(target_str)
dev = tvm.cpu(0)


print(f"Compiling TinyMLP...")
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod_mlp, target=target, params=params_mlp)
# Save compiled library
lib_name = f"TinyMLP_tvm.so"
lib.export_library(lib_name)
print(f"✅ TinyMLP compiled and exported as {lib_name}")


One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


Compiling TinyMLP...
✅ TinyMLP compiled and exported as TinyMLP_tvm.so
